## 라이브러리 설치

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 91.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 101.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 24.0 MB/s eta 0:00:00


## 모델 호출

In [61]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("wumusill/final_project_kogpt2")
model = AutoModelForCausalLM.from_pretrained("wumusill/final_project_kogpt2")

## 삼행시 함수

In [3]:
import torch

# GPU 사용 여부
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")

model.to(device)
model.eval()
print(device)

cpu


In [140]:
def mind(input_letter):
    # 두음 법칙 사전
    dooeum = {"라":"나", "락":"낙", "란":"난", "랄":"날", "람":"남", "랍":"납", "랑":"낭", 
          "래":"내", "랭":"냉", "냑":"약", "략":"약", "냥":"양", "량":"양", "녀":"여", 
          "려":"여", "녁":"역", "력":"역", "년":"연", "련":"연", "녈":"열", "렬":"열", 
          "념":"염", "렴":"염", "렵":"엽", "녕":"영", "령":"영", "녜":"예", "례":"예", 
          "로":"노", "록":"녹", "론":"논", "롱":"농", "뢰":"뇌", "뇨":"요", "료":"요", 
          "룡":"용", "루":"누", "뉴":"유", "류":"유", "뉵":"육", "륙":"육", "륜":"윤", 
          "률":"율", "륭":"융", "륵":"늑", "름":"늠", "릉":"능", "니":"이", "리":"이", 
          "린":'인', '림':'임', '립':'입'}
    # 결과물을 담을 list
    res_l = []

    # 한 글자씩 인덱스와 함께 가져옴
    for idx, val in enumerate(input_letter):
        # 두음 법칙 적용
        if val in dooeum.keys():
            val = dooeum[val]

        times = 0
        while times < 3:
            # 만약 idx 가 0 이라면 == 첫 글자
            if idx == 0:
                # 첫 글자 인코딩
                input_ids = tokenizer.encode(
                val, add_special_tokens=False, return_tensors="pt")
                
                # 첫 글자 인코딩 값으로 문장 생성
                output_sequence = model.generate(
                    input_ids=input_ids.to(device), 
                    do_sample=True, max_length=42,
                    min_length=5, temperature=0.9, repetition_penalty=1.5,
                    no_repeat_ngram_size=2)
            
            # 첫 글자가 아니라면
            else:
                # 좀더 매끄러운 삼행시를 위해 이전 문장이랑 현재 음절 연결
                # 이후 generate 된 문장에서 이전 문장에 대한 데이터 제거
                link_with_pre_sentence = " ".join(res_l) + " " + val  
                # print(link_with_pre_sentence)

                # 연결된 문장을 인코딩
                input_ids = tokenizer.encode(
                link_with_pre_sentence, add_special_tokens=False, return_tensors="pt")

                # 인코딩 값으로 문장 생성
                output_sequence = model.generate(
                    input_ids=input_ids.to(device), 
                    do_sample=True, max_length=42,
                    min_length=len_sequence + 5, temperature=0.9, repetition_penalty=1.5,
                    no_repeat_ngram_size=2)

            # 생성된 문장 리스트로 변환 (인코딩 되어있고, 생성된 문장 뒤로 padding 이 있는 상태)
            generated_sequence = output_sequence.tolist()[0]

            # padding index 앞까지 slicing 함으로써 padding 제거, padding이 없을 수도 있기 때문에 조건문 확인 후 제거
            if tokenizer.pad_token_id in generated_sequence:
                generated_sequence = generated_sequence[:generated_sequence.index(tokenizer.pad_token_id)]
            
            # 첫 글자가 아니라면, generate 된 음절만 결과물 list에 들어갈 수 있게 앞 문장에 대한 인코딩 값 제거
            # print(generated_sequence)
            if idx != 0:
                # 이전 문장의 길이 이후로 슬라이싱해서 앞 문장 제거
                generated_sequence = generated_sequence[len_sequence:]

                # 다음 음절을 위해 길이 갱신
                len_sequence += len(generated_sequence)        
            
            # 첫 글자라면
            else:
                # 시퀀스 길이 저장
                len_sequence = len(generated_sequence)

            # 결과물 디코딩
            decoded_sequence = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True)

            if len(decoded_sequence) > 1:
                break
            else:
                times += 1
                continue

        # 결과물 리스트에 담기
        res_l.append(decoded_sequence)

        # print(res_l)

    # 결과물 list에서 한 줄씩 출력
    for letter, res in zip(input_letter, res_l):
        print(f"{letter} :", res)

In [139]:
mind("배고파")

배 : 배트맨과 조커
고 : 고저 케인들
파 : 파랑 던들


In [134]:
mind("코코아")

코 : 코끝이 찡한 것을 느끼지
코 : 코 끝이 닿게
아 : 아련한 기억


In [107]:
mind("해파리")

해 : 해맑게 웃어주는 너
파 : 파노을처럼
리 : 이 길을 달려


In [108]:
mind("아이스크림")

아 : 아련하게 속삭이는
이 : 이 한마디
스 : 스쳐가는 인연이길
크 : 크겠어요
림 : 임을놓지 않는 아름다운 그대 그리운 말들은 사라져 가네 여기 서있는데 절대 뒤돌아보지마 하지만


In [132]:
mind("파이널")

파 : 파도에 사라져
이 : 이젠
널 : 널 잊어보려고 해


In [110]:
mind("박경택")

박 : 박혀있는 그 조각들을
경 : 경적해
택 : 택시 끝났어 첨으로 눈을 뗄 수 없었지


In [111]:
mind("문종현")

문 : 문득 그리운 맘에 전화를 걸죠
종 : 종일
현 : 현잔한 기분오던


In [112]:
mind("이정은")

이 : 이 밤이 새도록 널 기다려
정 : 정말이야
은 : 은빛은 우릴 돌려주네


In [113]:
mind("권소희")

권 : 권태기가 아냐 이건 끝 대체 어디부터 잘못된건지
소 : 소용이 없어
희 : 희미한 불빛


In [114]:
mind("박건영")

박 : 박수를 치자
건 : 건네주리
영 : 영영 차가운 시선


In [115]:
mind("정재영")

정 : 정처 없이 걷다가
재 : 재촉하는 낯선 발걸음은
영 : 영원이게 멀어지네


In [116]:
mind("구자현")

구 : 구겨져진 지난 기억
자 : 자꾸만
현 : 현아


In [117]:
mind("김의준")

김 : 김건모 노래
의 : 의자와
준 : 준담


In [130]:
mind("최지영")

최 : 최희 입니다
지 : 지친 맘을
영 : 영영 닫혀진 문 틈에서 난


In [131]:
mind("이지혜")

이 : 이제와서 비워야하니 자신이 없어
지 : 지우려 해도 만질 수 없는 걸
혜 : 혜오


# 혹시 몰라 저장
* 와일문 추가 전 스트림릿

In [ ]:
import requests
import streamlit as st
from streamlit_lottie import st_lottie
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Page Config
st.set_page_config(
    page_title="노래 가사 n행시",
    page_icon="💌",
    layout="wide"
)

### Model
tokenizer = AutoTokenizer.from_pretrained("wumusill/final_20man")

@st.cache(show_spinner=False)
def load_model():
    model = AutoModelForCausalLM.from_pretrained("wumusill/final_20man")
    return model

model = load_model()

# Class : Dict 중복 키 출력
class poem(object):
    def __init__(self,letter):
        self.letter = letter

    def __str__(self):
        return self.letter

    def __repr__(self):
        return "'"+self.letter+"'"


def n_line_poem(input_letter):

    # 두음 법칙 사전
    dooeum = {"라":"나", "락":"낙", "란":"난", "랄":"날", "람":"남", "랍":"납", "랑":"낭", 
          "래":"내", "랭":"냉", "냑":"약", "략":"약", "냥":"양", "량":"양", "녀":"여", 
          "려":"여", "녁":"역", "력":"역", "년":"연", "련":"연", "녈":"열", "렬":"열", 
          "념":"염", "렴":"염", "렵":"엽", "녕":"영", "령":"영", "녜":"예", "례":"예", 
          "로":"노", "록":"녹", "론":"논", "롱":"농", "뢰":"뇌", "뇨":"요", "료":"요", 
          "룡":"용", "루":"누", "뉴":"유", "류":"유", "뉵":"육", "륙":"육", "륜":"윤", 
          "률":"율", "륭":"융", "륵":"늑", "름":"늠", "릉":"능", "니":"이", "리":"이", 
          "린":'인', '림':'임', '립':'입'}
    # 결과물을 담을 list
    res_l = []

    # 한 글자씩 인덱스와 함께 가져옴
    for idx, val in enumerate(input_letter):
        # 두음 법칙 적용
        if val in dooeum.keys():
            val = dooeum[val]

        # 만약 idx 가 0 이라면 == 첫 글자
        if idx == 0:
            # 첫 글자 인코딩
            input_ids = tokenizer.encode(
            val, add_special_tokens=False, return_tensors="pt")
            
            # 첫 글자 인코딩 값으로 문장 생성
            output_sequence = model.generate(
                input_ids=input_ids, 
                do_sample=True, max_length=42, no_repeat_ngram_size=2,
                min_length=5, temperature=0.9, repetition_penalty=1.5)
        
        # 첫 글자가 아니라면
        else:
            # 좀더 매끄러운 삼행시를 위해 이전 문장이랑 현재 음절 연결
            # 이후 generate 된 문장에서 이전 문장에 대한 데이터 제거
            link_with_pre_sentence = " ".join(res_l) + " " + val  
            # print(link_with_pre_sentence)

            # 연결된 문장을 인코딩
            input_ids = tokenizer.encode(
            link_with_pre_sentence, add_special_tokens=False, return_tensors="pt")

            # 인코딩 값으로 문장 생성
            output_sequence = model.generate(
                input_ids=input_ids, 
                do_sample=True, max_length=42, no_repeat_ngram_size=2,
                min_length=len_sequence, temperature=0.9, repetition_penalty=1.5)

        # 생성된 문장 리스트로 변환 (인코딩 되어있고, 생성된 문장 뒤로 padding 이 있는 상태)
        generated_sequence = output_sequence.tolist()[0]

        # padding index 앞까지 slicing 함으로써 padding 제거, padding이 없을 수도 있기 때문에 조건문 확인 후 제거
        if tokenizer.pad_token_id in generated_sequence:
            generated_sequence = generated_sequence[:generated_sequence.index(tokenizer.pad_token_id)]
        
        # 첫 글자가 아니라면, generate 된 음절만 결과물 list에 들어갈 수 있게 앞 문장에 대한 인코딩 값 제거
        # print(generated_sequence)
        if idx != 0:
            # 이전 문장의 길이 이후로 슬라이싱해서 앞 문장 제거
            generated_sequence = generated_sequence[len_sequence:]

            # 다음 음절을 위해 길이 갱신
            len_sequence += len(generated_sequence)        
        
        # 첫 글자라면
        else:
            # 시퀀스 길이 저장
            len_sequence = len(generated_sequence)
        
        # print(last_sequence)

        # 결과물 디코딩
        decoded_sequence = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True)

        # 결과물 리스트에 담기
        res_l.append(decoded_sequence)

    poem_dict = {}

    for letter, res in zip(input_letter, res_l):
        poem_dict[poem(letter)] = res

    return poem_dict

###

# Image(.gif)
@st.cache(show_spinner=False)
def load_lottieurl(url: str):
    r = requests.get(url)
    if r.status_code != 200:
        return None
    return r.json()

lottie_url = "https://assets7.lottiefiles.com/private_files/lf30_fjln45y5.json"

lottie_json = load_lottieurl(lottie_url)
st_lottie(lottie_json, speed=1, height=200, key="initial")


# Title
row0_spacer1, row0_1, row0_spacer2, row0_2, row0_spacer3 = st.columns(
    (0.01, 2, 0.05, 1, 0.01)
)

with row0_1:
    st.title("한글 노래 가사 n행시")
    st.subheader("멋쟁이사자처럼 AIS7 파이널 프로젝트")

with row0_2:
    st.write("")
    st.subheader(
        "해파리팀"
    )
    st.write("이지혜, 최지영, 권소희")
    st.write("문종현, 구자현, 김의준")

st.write('---')

# Explanation
row1_spacer1, row1_1, row1_spacer2 = st.columns((0.01, 3, 0.01))

with row1_1:
    st.markdown(
        "**'MZ세대'에게**"
    )
    st.markdown(
        "음악은 세대를 드러내는 지표이자 자신의 감정 및 공동체를 드러내는 수단이다."
    )

st.write('---')

# Model & Input
row2_spacer1, row2_1, row2_spacer2 = st.columns((0.01, 1.5, 0.05))

# Word Input
if "generate" not in st.session_state:
    st.session_state.generate = False

with row2_1:
    word_input = st.text_input(
            "n행시에 사용할 단어를 적고 Enter를 눌러주세요. 👇",
            placeholder='한글 단어',
            max_chars=10
    )
    
    if word_input:
        st.write("n행시 단어 :  ", word_input)

    if st.button('n행시 제작하기'):
        with st.spinner('잠시 기다려주세요...'):
            result = n_line_poem(word_input)
        st.success('완료됐습니다!')
        for r in result:
            st.write(f'{r} : {result[r]}')